In [1]:
import pandas as pd
data=pd.read_csv("train.csv")
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [2]:
data=data.dropna()
data.reset_index(inplace=True)

In [3]:
X = data.drop("label",axis=1)
y = data["label"]

In [4]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

C:\Users\Abhinav Raja Raizada\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


###Preprocessing The Text 



In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Abhinav Raja
[nltk_data]     Raizada\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # (word not in stopwords) and stem(word)
  review = ' '.join(review)
  corpus.append(review)

### One Hot Encoding

In [8]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 

###Applying Pad Sequences To Make Sentence Length Equal

In [9]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4462 3966 2014]
 [   0    0    0 ... 2968 3843  736]
 [   0    0    0 ... 3251 2670 2532]
 ...
 [   0    0    0 ... 4805 1362 1826]
 [   0    0    0 ...  124 3372  300]
 [   0    0    0 ... 4944 1589 4096]]


#Creating Model for Train Set

In [10]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [13]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 11s 37ms/step - loss: 0.3416 - accuracy: 0.8398 - val_loss: 0.1973 - val_accuracy: 0.9200
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1442 - accuracy: 0.9440 - val_loss: 0.2081 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0972 - accuracy: 0.9657 - val_loss: 0.2111 - val_accuracy: 0.9147
Epoch 4/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0769 - accuracy: 0.9733 - val_loss: 0.2517 - val_accuracy: 0.9059
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0554 - accuracy: 0.9821 - val_loss: 0.2921 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0402 - accuracy: 0.9874 - val_loss: 0.3149 - val_accuracy: 0.9089
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0289 - accuracy: 0.9917 - val_loss: 0.3730 - val_accuracy: 0.914

###Performance Metrics And Accuracy



In [14]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

189/189 [==============================] - 2s 8ms/step


array([[3110,  309],
       [ 221, 2395]], dtype=int64)

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9121789560894781